<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/marco-canas/Machine-Learning/blob/main/ML/classes/class_feb_29/class_feb_29.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table>

# Construcción de un modelo de regresión lineal

4. Preprocesamiento usando el concepto de pipeline  
   * Hacer separación inicial de datos para entrenar y para testear.
   * llenar datos faltantes.
   * adicionar atributos que estén mejor correlacionados con la variable objetivo.
   * Estandarizar los datos.
   * codificar las variables categóricas. 
     

5. Entrenar el modelo de regresión lineal.

   

# Transformadores personalizados

Aunque Scikit-Learn proporciona muchos transformadores útiles, deberá escribir los suyos propios para tareas como:

* operaciones de limpieza personalizadas o 
* combinación de atributos específicos.

 
* todo lo que necesita hacer es crear una clase e implementar tres métodos: 
   * `fit( )` (retornando a sí mismo), 
   * `transform()` y 
   * `fit_transform()`.

Puede obtener el último de forma gratuita simplemente agregando `TransformerMixin` como clase base. 

Si agrega `BaseEstimator` como clase base (y evita `*args` y `**kargs` en su constructor), también obtendrá dos métodos adicionales (`get_params()` y `set_params()`) que son útiles para el ajuste automático de hiperparámetros.

Por ejemplo, aquí hay una pequeña clase de transformador que agrega los atributos combinados que discutimos anteriormente:

## Importar las liberías necesarias

In [ ]:
import pandas as pd 
import numpy as np 

from sklearn.model_selection import train_test_split 

from sklearn.base import BaseEstimator, TransformerMixin


In [1]:
v = pd.read_csv('vivienda.csv') 

In [2]:
v.head(1) 

,longitud,latitud,antiguedad,habitaciones,dormitorios,población,hogares,ingresos,proximidad,precio
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,NEAR BAY,452600.0


In [ ]:

v_train, v_test = train_test_split(v, test_size = 0.2, random_state = 513) 

## Dividir entre predictores y objetivo

In [3]:
v = v_train.drop('precio', axis = 1) 
v_labels = v_train.precio 

In [5]:
class AdAtribComb(BaseEstimator, TransformerMixin):
    def __init__(self, ad_dph = True, ad_hph = True, ad_pph = True): # no *args or **kargs
        self.ad_dph = ad_dph
        self.ad_hph = ad_hph
        self.ad_pph = ad_pph
        
    def fit(self, X, y=None):
        return self # nothing else to do
    
    def transform(self, X, y=None):
        habitaciones, dormitorios, población, hogares = 3, 4, 5, 6 
        if self.ad_dph:
            dormitorios_por_habitación=X[:,dormitorios]/X[:,habitaciones]
            X=np.c_[X, dormitorios_por_habitación]
        if self.ad_hph:
            habitaciones_por_hogar=X[:,habitaciones]/X[:,hogares]
            X=np.c_[X, habitaciones_por_hogar]
        if self.ad_pph: 
            población_por_hogar = X[:, población] / X[:, hogares]
            X=np.c_[X, población_por_hogar] 
        return X
       

In [6]:
adicionador_atributos = AdAtribComb(ad_dph=True, ad_hph=False, ad_pph=False)

In [7]:
X = adicionador_atributos.fit_transform(v.values)   
X.shape 

(16512, 10)

In [ ]:
def recuperar_df(X):
    columnas = list(v.columns) 
    if adicionador_atributos.ad_dph: 
        columnas += ['dormitorios_por_habitación']
    if adicionador_atributos.ad_hph:
        columnas += ['habitaciones_por_hogar'] 
    if adicionador_atributos.ad_pph:
        columnas += ['población_por_hogar'] 
    indices = v.index
    return pd.DataFrame(X, columns = columnas, index = indices) 

In [ ]:
df = recuperar_df(X,ad_dph=True,ad_hph=False, ad_pph=False)
df.head(2)

En este ejemplo, el transformador tiene tres hiperparámetros:

* ad_dph, 
* ad_hph y
* ad_pph,

establecidos en `True` de forma predeterminada (a menudo es útil proporcionar valores predeterminados sensibles).

Estos hiperparámetros le permitirá averiguar fácilmente si agregar estos atributos ayudan o no a los algoritmos de Machine Learning.

En términos más generales, puede agregar un hiperparámetro para controlar cualquier paso de preparación de datos del que no esté $100 \%$ seguro.

Cuanto más automatice estos pasos de preparación de datos, más combinaciones podrá probar automáticamente, lo que hará que sea mucho más probable que encuentre una gran combinación (y le ahorrará mucho tiempo).

## Escalado de características

Una de las transformaciones más importantes que debe aplicar a sus datos es el escalado de atributos. 

Con pocas excepciones, los algoritmos de Machine Learning no funcionan bien cuando los atributos numéricos de entrada tienen escalas muy diferentes.

Este es el caso de los datos de vivienda: 

* el número total de habitaciones oscila entre unas 6 y 39.320, 
* mientras que los ingresos medios solo oscilan entre 0 y 15.

Tenga en cuenta que, por lo general, no es necesario escalar los valores objetivo.

Hay dos formas comunes de hacer que todos los atributos tengan la misma escala: 
* escala minmax y 
* estandarización.

El escalado mínimo-máximo (muchas personas llaman a esto normalización) es el más simple:  

* los valores se desplazan y reescalan para que terminen variando de 0 a 1.

Hacemos esto restando el valor mínimo y dividiendo por el máximo menos el mínimo.

Scikit-Learn proporciona un transformador llamado `MinMaxScaler` para esto.

Tiene un hiperparámetro `feature_range` que le permite cambiar el rango si, por alguna razón, no desea 0–1.

In [ ]:
v_num = v.drop('proximidad', axis = 1)

In [ ]:
from sklearn.preprocessing import MinMaxScaler 

In [ ]:
escalar_min_max = MinMaxScaler() 

In [ ]:
X_escalado_min_max = escalar_min_max.fit_transform(v_num)

In [ ]:
v_escalado_min_max = pd.DataFrame(X_escalado_min_max, columns = v_num.columns, \
                                  index = v_num.index)

In [ ]:
v_escalado_min_max.head() 

In [ ]:
v_escalado_min_max.describe() 

In [ ]:
v_num.min() # Si se aplica una función a un DataFrame, esta actuará por columnas. 

In [ ]:
v_num.apply(lambda df:(df-df.min())/(df.max() - df.min()))

# La estandarización de datos 

La estandarización es diferente:  

* primero resta el valor medio (por lo que los valores estandarizados siempre tienen una media cero) y luego 
* divide por la desviación estándar para que la distribución resultante tenga varianza unitaria.

A diferencia del escalado mínimo-máximo, la estandarización no vincula los valores a un rango específico, lo que puede ser un problema para algunos algoritmos (por ejemplo, las redes neuronales a menudo esperan un valor de entrada de 0 a 1).

Sin embargo, la estandarización se ve mucho menos afectada por los valores atípicos.

Por ejemplo, suponga que un distrito tiene un ingreso medio igual a 100 (por error). La escala min-max aplastaría todos los demás valores de 0 a 15 hasta 0 a 0,15, mientras que la estandarización no se vería muy afectada. ScikitLearn proporciona un transformador llamado `StandardScaler` para la estandarización.

## ADVERTENCIA

Al igual que con todas las transformaciones, es importante ajustar los escaladores solo a los datos de entrenamiento, no al conjunto de datos completo (incluido el conjunto de prueba).

Solo entonces puede usarlos para transformar el conjunto de entrenamiento y el conjunto de prueba (y nuevos datos).

## Transformation Pipelines

Como puede ver, hay muchos pasos de transformación de datos que deben ejecutarse en el orden correcto.

Afortunadamente, Scikit-Learn proporciona la clase `Pipeline` para ayudar con tales secuencias de transformaciones.

Aquí hay una pequeña tubería para los atributos numéricos:

In [ ]:
import numpy as np 
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer 
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import train_test_split 

In [ ]:
v = pd.read_csv('vivienda.csv') 
v_train, v_test = train_test_split(v, test_size = 0.2, random_state = 513)
v = v_train
v_num = v.drop(['proximidad', 'precio'], axis = 1)

In [ ]:
habitaciones, dormitorios, población, hogares = 3, 4, 5, 6 
class AdAtribComb(BaseEstimator, TransformerMixin):
    def __init__(self, ad_dph = True, ad_hph = True, ad_pph = True): # no *args or **kargs
        self.ad_dph = ad_dph
        self.ad_hph = ad_hph
        self.ad_pph = ad_pph
        
    def fit(self, X, y=None):
        return self # nothing else to do
    def transform(self, X, y=None):
        if self.ad_dph:
            dormitorios_por_habitación = X[:, dormitorios] / X[:, habitaciones]
            X=np.c_[X, dormitorios_por_habitación]
        if self.ad_hph:
            habitaciones_por_hogar = X[:, habitaciones] / X[:, hogares]
            X=np.c_[X, habitaciones_por_hogar]
        if self.ad_pph: 
            población_por_hogar = X[:, población] / X[:, hogares]
            X=np.c_[X, población_por_hogar] 
        return X
       

In [ ]:
pipeline_num = Pipeline([ ('imputador', SimpleImputer(strategy = 'median')),
                        ('adicionar_atributos', AdAtribComb()),
                        ('estandarizar', StandardScaler()) 
                         ])

In [ ]:
X_num_tr = pipeline_num.fit_transform(v_num) 

In [ ]:
v_num_tr = pd.DataFrame(X_num_tr, columns = list(v_num.columns) + \
                        ['dormitorios_por_habitación', 'habitaciones_por hogar', \
                        'población_por_hogar'], \
                        index = v_num.index)
v_num_tr.head() 

In [ ]:
v_num_tr.info()

In [ ]:
v_num_tr.describe() 

El constructor Pipeline toma una lista de pares de nombre/estimador que definen una secuencia de pasos.

Todos menos el último estimador deben ser transformadores (es decir, deben tener un método `fit_transform()`).

Los nombres pueden ser cualquier cosa que desee (siempre que sean únicos y no contengan guiones bajos dobles, __); serán útiles más adelante para el ajuste de hiperparámetros.

Cuando llamas al método `fit()` de la canalización, este llama a `fit_transform()` secuencialmente en todos los transformadores, pasando la salida de cada llamada como parámetro a la siguiente llamada hasta que llega al estimador final, para lo cual llama al método `fit()`.

La canalización expone los mismos métodos que el estimador final.

En este ejemplo, el último estimador es un `StandardScaler`, que es un transformador, por lo que la canalización tiene un método `transform()` que aplica todas las transformaciones a los datos en secuencia (y, por supuesto, también un método `fit_transform()`, que es el que usamos).

Hasta ahora, hemos manejado las columnas categóricas y las columnas numéricas por separado.

Sería más conveniente tener un solo transformador capaz de manejar todas las columnas, aplicando las transformaciones adecuadas a cada columna.

En la versión 0.20, Scikit-Learn introdujo `ColumnTransformer` para este propósito, y la buena noticia es que funciona muy bien con pandas DataFrames.

Let’s use it to apply all the transformations to the housing data:

In [ ]:
from sklearn.compose import ColumnTransformer
lista_atributos_num = list(v_num.columns)
lista_atributos_cat = ['proximidad']
full_pipeline = ColumnTransformer([
                                   ("numerica", pipeline_num, lista_atributos_num),
                                   ("categórica", OneHotEncoder(), lista_atributos_cat),
                                  ])
X_preparado = full_pipeline.fit_transform(v)

In [ ]:
X_preparado.shape 

In [ ]:
def recuperar_df(X,ad_dph=True,ad_hph=False, ad_pph=False):
    columnas = list(v_num.columns) 
    if ad_dph: 
        columnas += ['dormitorios_por_habitación']
    if ad_hph:
        columnas += ['habitaciones_por_hogar'] 
    if ad_pph:
        columnas += ['población_por_hogar'] 
    columnas+=['hora', 'interior','isla', 'bahia', 'oceano' ]    
    indices = v.index
    return pd.DataFrame(X, columns = columnas, index = indices) 

In [ ]:
v_cat = v[['proximidad']]

In [ ]:
codificador = OneHotEncoder()

In [ ]:
X_cat = codificador.fit_transform(v_cat)

In [ ]:
codificador.categories_

In [ ]:
v_preparado = recuperar_df(x_preparado, ad_dph = True, ad_hph = True, ad_pph = True ) 

In [ ]:
v_preparado.info() 

Primero importamos la clase `ColumnTransformer`, luego obtenemos la lista de nombres de columnas numéricas y la lista de nombres de columnas categóricas, y luego construimos un `ColumnTransformer`.

El constructor requiere una lista de tuplas, donde cada tupla contiene un nombre, un transformador y una lista de nombres (o índices) de columnas a las que se debe aplicar el transformador.

En este ejemplo, especificamos que las columnas numéricas deben transformarse usando num_pipeline que definimos anteriormente, y las columnas categóricas deben transformarse usando un `OneHotEncoder`.

Finalmente, aplicamos este ColumnTransformer a los datos de la vivienda: aplica cada transformador a las columnas apropiadas y concatena las salidas a lo largo del segundo eje (los transformadores deben devolver el mismo número de filas).

Tenga en cuenta que `OneHotEncoder` devuelve una **matriz dispersa**, mientras que `num_pipeline` devuelve una **matriz densa**.

Cuando existe tal combinación de matrices densas y dispersas, ColumnTransformer estima la densidad de la matriz final (es decir, la proporción de celdas distintas de cero) y devuelve una matriz dispersa si la densidad es inferior a un umbral dado (por defecto, umbral_disperso=0.3).

En este ejemplo, devuelve una matriz densa.

¡Y eso es! Tenemos una canalización de preprocesamiento que toma los datos de vivienda completos y aplica las transformaciones apropiadas a cada columna.

## TIP

Instead of using a transformer, you can specify the string "drop" if you want the columns to
be dropped, or you can specify "passthrough" if you want the columns to be left untouched.

By default, the remaining columns (i.e., the ones that were not listed) will be dropped, but
you can set the remainder hyperparameter to any transformer (or to "passthrough") if you
want these columns to be handled differently.

If you are using Scikit-Learn 0.19 or earlier, you can use a third-party library such as sklearn-pandas, or you can roll out your own custom transformer to get the same functionality as the ColumnTransformer. 

Alternatively, you can use the FeatureUnion class, which can apply different transformers and
concatenate their outputs. 

But you cannot specify different columns for each transformer; they all apply to the whole data. 

It is possible to work around this limitation using a custom transformer for column selection (see the Jupyter notebook for an example).

## Select and Train a Model

¡Por fin! 

* Enmarcó el problema, 
* obtuvo los datos y los exploró, 
* muestreó un conjunto de entrenamiento y un conjunto de prueba, y 
* escribió canalizaciones de transformación para limpiar y preparar sus datos para los algoritmos de aprendizaje automático automáticamente.

Ahora está listo para seleccionar y entrenar un modelo de Machine Learning.

## Entrenamiento y evaluación en el conjunto de entrenamiento

The good news is that thanks to all these previous steps, things are now going to be much simpler than you might think. 

Let’s first train a Linear Regression model, like we did in the previous chapter:


In [ ]:
from sklearn.linear_model import LinearRegression
reg_lin = LinearRegression()
reg_lin.fit(v_preparado, v_labels)

¡Hecho! Ahora tiene un modelo de regresión lineal en funcionamiento. Probémoslo en algunas instancias del conjunto de entrenamiento:

In [ ]:
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)
print("Predictions:", lin_reg.predict(some_data_prepared))

In [ ]:
print("Labels:", list(some_labels))

## [Video de apoyo](https://www.youtube.com/watch?v=o-GSsrfHmEk)

# Randomized Search

El enfoque de búsqueda en cuadrícula está bien cuando está explorando relativamente pocas combinaciones, como en el ejemplo anterior, pero cuando el espacio de búsqueda de hiperparámetros es grande, a menudo es preferible usar `RandomizedSearchCV` en su lugar.

Esta clase se puede usar de la misma manera que la clase `GridSearchCV`, pero en lugar de probar todas las combinaciones posibles, evalúa un número determinado de combinaciones aleatorias seleccionando un valor aleatorio para cada hiperparámetro en cada iteración.

This approach has two main benefits: If you let the randomized search run for, say, 1,000 iterations, this
approach will explore 1,000 different values for each hyperparameter (instead of just a few values per hyperparameter with the grid search approach).

Simply by setting the number of iterations, you have more control over the computing budget you want to allocate to hyperparameter search.

## Ensemble Methods


Otra forma de ajustar su sistema es intentar combinar los modelos que funcionan mejor.

The group (or “ensemble”) will often perform better than the best individual model (just like Random Forests perform better than the individual Decision Trees they rely on), especially if the individual models make very different types of errors. 

We will cover this topic in more detail in Chapter 7.

Let’s first train a Linear Regression model, like we did in the previous chapter:

In [ ]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

Done! You now have a working Linear Regression model. 

Let’s try it out on a few instances from the training set:

In [ ]:
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)
print("Predictions:", lin_reg.predict(some_data_prepared))


In [ ]:
print("Labels:", list(some_labels))


It works, although the predictions are not exactly accurate (e.g., the first prediction is off by close to 40%!). 

Let’s measure this regression model’s RMSE on the whole training set using Scikit-Learn’s mean_squared_error() function:

In [ ]:
from sklearn.metrics import mean_squared_error
housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse


This is better than nothing, but clearly not a great score: most districts’ median_housing_values range between $120,000 and $265,000, so a typical prediction error of $68,628 is not very satisfying. 

This is an example of a model underfitting the training data. 

When this happens it can mean that the features do not provide enough information to make good predictions, or that the model is not powerful enough. 

As we saw in the previous chapter, the main ways to fix underfitting are to select a more powerful model, to feed the training algorithm with better features, or to reduce the constraints on the model. 

This model is not regularized, which rules out the last option. 

You could try to add more features (e.g., the log of the population), but first let’s try a more complex model to see how it does.

Let’s train a DecisionTreeRegressor. 

This is a powerful model, capable of finding complex nonlinear relationships in the data (Decision Trees are presented in more detail in Chapter 6). 

The code should look familiar by now:

## Referentes

* La clase Pipeline: https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html